In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

haiku_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are specialized in writing haikus about {topic}."),
    ("human", "I want to write a haiku about {topic}")
])

explain_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are specialized in explaining a haiku"),
    ("human", "{haiku}")
])

haiku_chain = haiku_prompt | chat
explain_chain = explain_prompt | chat

final_chain = { "haiku": haiku_chain } | explain_chain

final_chain.invoke({"topic": "Python"})


Languages of code,
Syntax and logic combine,
Creating magic.This haiku beautifully captures the essence of programming. It describes how different programming languages, each with its own syntax and logic, come together to create something magical. The poet is highlighting the creative and almost mystical process of coding, where lines of code transform into functional and sometimes awe-inspiring software. The haiku celebrates the art and science of programming, emphasizing the power and beauty that can be found in the world of coding.

AIMessage(content='This haiku beautifully captures the essence of programming. It describes how different programming languages, each with its own syntax and logic, come together to create something magical. The poet is highlighting the creative and almost mystical process of coding, where lines of code transform into functional and sometimes awe-inspiring software. The haiku celebrates the art and science of programming, emphasizing the power and beauty that can be found in the world of coding.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-0b50f5ea-cc68-4f62-8886-a96abfc2d983-0')